In [1]:
import numpy as np
import sympy as sp
sp.init_printing()

from manipylator.utils import parametric_heart_1, parametric_heart_2

### `parametric_heart_1` & `parameteric_heart_2` reproduced for clarity

https://mathworld.wolfram.com/HeartCurve.html

```python
def parametric_heart_1(t):
    x = np.sqrt(2) * np.sin(t) ** 3
    y = -np.cos(t) ** 3 - np.cos(t) ** 2 + 2 * np.cos(t)

    return x, y


def parametric_heart_2(t):
    x = (16 * np.sin(t) ** 3) / 11
    y = (
        13 * np.cos(t) - 5 * np.cos(2 * t) - 2 * np.cos(3 * t) - np.cos(4 * t) - 5
    ) / 11

    return x, y
```

In [2]:
t = sp.symbols('t')
x, y = sp.symbols('x y', cls=sp.Function)

In [3]:
x = sp.sqrt(2) * sp.sin(t) ** 3
y = -sp.cos(t) ** 3 - sp.cos(t) ** 2 + 2 * sp.cos(t)

In [4]:
x

In [5]:
y

In [6]:
i = sp.eye(4)
# i[2,2] = 0
# i

Parametric path in matrix form

In [7]:
i[0,3] = x
i[1,3] = y
i

⎡                         3             ⎤
⎢1  0  0            √2⋅sin (t)          ⎥
⎢                                       ⎥
⎢              3         2              ⎥
⎢0  1  0  - cos (t) - cos (t) + 2⋅cos(t)⎥
⎢                                       ⎥
⎢0  0  1                0               ⎥
⎢                                       ⎥
⎣0  0  0                1               ⎦

Numeric pose in matrix form

In [8]:
f_i = sp.lambdify(t, i, 'numpy')
f_i(1)

array([[1.        , 0.        , 0.        , 0.8426213 ],
       [0.        , 1.        , 0.        , 0.63094942],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [9]:
points = np.linspace(0, 2*np.pi, 100)
# points

In [10]:
positions = [f_i(t) for t in points]
# positions

In [11]:
positions[3]

array([[1.        , 0.        , 0.        , 0.00958586],
       [0.        , 1.        , 0.        , 0.05291352],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

extracting only x,y positions

In [12]:
xy = [(p[0,3], p[1,3]) for p in positions]
# print(xy)

In [15]:
import pandas as pd
import hvplot.pandas
hvplot.extension('bokeh')

Hilighting a specific point in the trajectory

In [16]:
df = pd.DataFrame(xy, columns=['x', 'y'])
df['t'] = points
df['color'] = 'blue'
df.loc[df.query('x<1.05 and x>0.99').index, 'color'] = 'red'
df.query('color == "red"')

,x,y,t,color
32,1.017255,-0.997761,2.030929,red


In [17]:
df.head()

,x,y,t,color
0,0.000000,0.000000,0.000000,blue
1,0.000361,0.006024,0.063467,blue
2,0.002869,0.023877,0.126933,blue
3,0.009586,0.052914,0.190400,blue
4,0.022403,0.092078,0.253866,blue


In [18]:
heart_plot = df.hvplot.scatter(x='x', y='y', color='color').opts(frame_width=400, frame_height=400,)
heart_plot

:Scatter   [x]   (y,color)

drawing tangent to a line

In [27]:
diff = df[['x', 'y']].diff()
df['m'] = diff.y/diff.x

In [20]:
p = df.loc[32]
p

x        1.017255
y       -0.997761
t        2.030929
color         red
m        1.443724
Name: 32, dtype: object

In [21]:
x0, x1 = 0.5, 1.5
y0, y1 = p.y - (p.x-x0)*p.m, p.y + (x1-p.x)*p.m

In [22]:
[(x0, y0), (x1, y1)]

In [23]:
line_df = pd.DataFrame([(x0, y0), (x1, y1)], columns=['x', 'y'])
line_plot = line_df.hvplot.line(x='x', y='y', color='red')

In [24]:
heart_plot*line_plot

:Overlay
   .Scatter.I :Scatter   [x]   (y,color)
   .Curve.I   :Curve   [x]   (y)